In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output
from tqdm import tqdm_notebook as tqdm
from torch import optim
import pandas as pd
import time
from torchvision import transforms, datasets
import torch.nn.functional as F
import os
from os import listdir
from os.path import isfile, join
import random

# Arquitectura

In [2]:
class ExpandedCnnLSTM(nn.Module):
  
  def __init__(self, resnet_model, n_channels, output_size, hidden_dim, n_lstm_layers, drop_prob=0.5, debug=False):

    super(ExpandedCnnLSTM, self).__init__()

    self.output_size = output_size
    self.n_lstm_layers = n_lstm_layers
    self.hidden_dim = hidden_dim
    self.n_channels = n_channels
          
    model = torch.hub.load('pytorch/vision:v0.6.0', resnet_model, pretrained=True)
    out_channels = model.conv1.in_channels
    kernel_size = model.conv1.kernel_size
    stride = model.conv1.stride
    padding = model.conv1.padding
    self.convPadd = nn.Conv2d(n_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
    
    self.embeding_dim = model.fc.in_features
    self.cnn = torch.nn.Sequential(*(list(model.children())[:-1]))
    if debug:
      print(model.eval())
      print(self.cnn[0].in_channels)
    

    self.lstm = nn.LSTM(self.embeding_dim, hidden_dim, n_lstm_layers, dropout=drop_prob, batch_first=True)
    self.dropout = nn.Dropout(drop_prob)
    self.fc = nn.Linear(hidden_dim, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):

    batch_size = x.size(0)
    n_frames = x.size(1)

    features = torch.zeros((batch_size, n_frames, self.embeding_dim))

    for i in range(n_frames):
        padded = self.convPadd(x[:,i,:])
        features[:,i,:] = self.cnn(padded).squeeze()

    lstm_out, hidden = self.lstm(features.cuda())
    lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
    out = self.dropout(lstm_out)
    out = self.fc(out)
    out = self.sigmoid(out)
    
    out = out.view(batch_size, -1)
    out = out[:,-1]
    
    return out

  def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_lstm_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_lstm_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [3]:
#cl_model = CnnLSTM(4, 1, 512, 2)
cl_model = ExpandedCnnLSTM('resnet18', 4, 1, 512, 2, debug=True)

Using cache found in /home/jupyter/.cache/torch/hub/pytorch_vision_v0.6.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
positive_path = '/home/jupyter/data-step/processed_data/positive/dataset/'
negative_path = '/home/jupyter/data-step/processed_data/negative/dataset/'

In [5]:
positive_path_norm = '/home/jupyter/data-step/normalized_data/positive/dataset/'
negative_path_norm = '/home/jupyter/data-step/normalized_data/negative/dataset/'

In [6]:
positive_files = [positive_path_norm + f for f in listdir(positive_path_norm)]
y_p = np.ones((len(positive_files)))
print(len(positive_files))

negative_files = [negative_path_norm + f for f in listdir(negative_path_norm)]
y_n = np.zeros((len(negative_files)))
print(len(negative_files))

1600
1184


In [7]:
total_files = positive_files + negative_files

y_tot = np.append(y_p, y_n)

print(len(total_files), y_tot.shape)

2784 (2784,)


In [8]:
def channel_metrics(data):
    n_channels = len(data[0,0, :])
    #print("num channels: ", n_channels)
    means = []
    stds = []

    for i in range(n_channels):
            means.append(np.mean(data[:, :, i, :, :]))
            stds.append(np.std(data[:, :, i, :, :]))
    #print(means, stds)
    return means, stds




def total_metrics(files):
    means = []
    stds = []
    count = 0
    
    for f in tqdm(files):
        #print(count, end='\r')
        data = torch.load(f)
        #print(data.shape)
        mean, std = channel_metrics(data.numpy())
        means.append(mean)
        stds.append(std)
        count += 1
        
    return means, stds

#means_p, stds_p = total_metrics(positive_files)
#means_n, stds_n = total_metrics(negative_files)
#means, stds = total_metrics(total_files)

In [9]:
#means_np = np.array(means)
#print(means_np.shape)

#stds_np = np.array(stds)
#print(stds_np.shape)

In [10]:
#means_np = np.append(np.array(means_p),np.array(means_n), axis=0)
#print(means_np.shape)

#stds_np = np. append(np.array(stds_p), np.array(stds_n), axis=0)
#print(stds_np.shape)

In [11]:
#norm_means = np.mean(means_np, axis=0)
#norm_stds = np.sqrt(np.sum((np.array(stds_np)**2), axis=0))/len(total_files)

#print(norm_means.shape, norm_stds.shape)
#print(norm_means, norm_stds)
#np.save("/home/jupyter/means.npy", norm_means)
#np.save("/home/jupyter/stds.npy", norm_stds)



#stds_np

In [12]:
norm_means = torch.tensor(np.load("means.npy")).float()
norm_stds = torch.tensor(np.load("stds.npy")).float()

In [13]:
preprocess = transforms.Compose([
                                transforms.Normalize(mean=norm_means, std=norm_stds)
                                ])

In [14]:
class LandslideDataset(Dataset):
  
    def __init__(self, dirs, y):

        self.dirs = sorted(dirs) 
        self.l = 5#len(self.dirs)
        self.y = y

    def __len__(self): 
        return len(self.y)*5   # REVISAR ESTO

    def __getitem__(self, idx):
        x = torch.load(self.dirs[int(idx/5)])[idx%5]
        for i in range(9):
            for c in range(16):
                #print("DL")
                x[i, c] = (x[i,c] - norm_means[c])/norm_stds[c]
            #x[i] = (x[i] - norm_means) / norm_stds
            #x[i] = preprocess(x[i].to(torch.float))
                
        return x, self.y[int(idx/5)]

In [25]:
class SateliteDataset(Dataset):
    
    def __init__(self, dirs, y):
        t = torch.tensor([])
        for file in dirs:
            t = torch.cat((t, torch.load(file)), 0)
            
        self.x = t#[torch.load(file)[j].cpu() for file in dirs for j in range(160)]
        self.y = y

    def __len__(self): 
        return len(self.y)*160   # REVISAR ESTO

    def __getitem__(self, idx):
                
        return self.x[idx], self.y[int(idx/160)]

In [ ]:
class DuoSateliteDataset(Dataset):

    def init(self, local_dirs, mounted_dirs):
        local_dirs         # contains train_files, train_labels
        mounted_dirs #  contains train_files, train_labels
        # local loading
        t_local = torch.tensor([])
        for file in local_dirs:
            t_local = torch.cat((t_local, torch.load(file))[:, -3:, ...], 0)

        # remote loading
        t_remote = torch.tensor([])
        for file in mounted_dirs:
            tmp_tensor = torch.load(file)[:, -3:, -1, ...]
            t_remote = torch.cat((t_remote, tmp_tensor), 0)

        self.x = torch.cat((t_local, t_remote), 2) #[torch.load(file)[j].cpu() for file in dirs for j in range(160)]
        self.y = y

    def len(self): 
        return len(self.y)*160   # REVISAR ESTO

    def getitem(self, idx):

        return self.x[idx], self.y[int(idx/160)]



def get_ds_dl(base_dir):
  pos_sat_files= [base_dir.replace('dummy', 'positive') + x  for x in os.listdir(pos_satelite_path)]

  neg_sat_files = [base_dir.replace('dummy', 'positive') + x  for x in os.listdir(neg_satelite_path)]

  random.shuffle(pos_sat_files)
  random.shuffle(neg_sat_files)

  pos_cut = int(0.9len(pos_sat_files))
  neg_cut = int(0.9*len(neg_sat_files))

  train_files = pos_sat_files[:pos_cut] + neg_sat_files[:neg_cut]
  train_labels = np.append(np.ones(pos_cut), np.zeros(neg_cut))

  val_files = pos_sat_files[pos_cut:] + neg_sat_files[neg_cut:]
  val_labels = np.append(np.ones(len(pos_sat_files) - pos_cut), np.zeros(len(neg_sat_files) - neg_cut))

 return [train_files, train_labels], [val_files, val_labels]


remote_train, remote_validation = get_ds_dl( '/home/jupyter/data-step/processed_data/dummy/cfs/' )
local_train, local_validation = get_ds_dl( '/home/jupyter/chirps/dummy/' )

train_ds = DuoSateliteDataset(local_train, remote_train)

In [23]:
#total_ds = LandslideDataset(total_files, y_tot)
random.shuffle(positive_files)
random.shuffle(negative_files)

print(positive_files[0])

pos_cut = int(0.9*len(positive_files))
neg_cut = int(0.9*len(negative_files))

train_files = positive_files[:pos_cut] + negative_files[:neg_cut]
train_labels = np.append(np.ones(pos_cut), np.zeros(neg_cut))
                         
val_files = positive_files[pos_cut:] + negative_files[neg_cut:]
val_labels = np.append(np.ones(len(positive_files) - pos_cut), np.zeros(len(negative_files) - neg_cut))

# Datasets
                         
train_ds = LandslideDataset(train_files, train_labels)
val_ds = LandslideDataset(val_files, val_labels)

/home/jupyter/data-step/normalized_data/positive/dataset/divided_part_7840_6.pt


In [17]:
# Dataloaders

train_dl = DataLoader(train_ds, num_workers=32)
val_dl = DataLoader(val_ds, num_workers=32)

In [18]:
pos_satelite_path = '/home/jupyter/chirps/positive/'
#print(listdir(pos_satelite_path))
pos_sat_files = [pos_satelite_path + x  for x in listdir(pos_satelite_path)]
print(len(pos_sat_files))

neg_satelite_path = '/home/jupyter/chirps/negative/'
#print(listdir(pos_satelite_path))
neg_sat_files = [neg_satelite_path + x  for x in listdir(neg_satelite_path)]

50


In [26]:
random.shuffle(pos_sat_files)
random.shuffle(neg_sat_files)

print(pos_sat_files[0])

pos_cut = int(0.9*len(pos_sat_files))
neg_cut = int(0.9*len(neg_sat_files))

train_files = pos_sat_files[:pos_cut] + neg_sat_files[:neg_cut]
train_labels = np.append(np.ones(pos_cut), np.zeros(neg_cut))
                         
val_files = pos_sat_files[pos_cut:] + neg_sat_files[neg_cut:]
val_labels = np.append(np.ones(len(pos_sat_files) - pos_cut), np.zeros(len(neg_sat_files) - neg_cut))

# Datasets
                         
train_sat_ds = SateliteDataset(train_files, train_labels)
val_sat_ds = SateliteDataset(val_files, val_labels)

train_sat_dl = DataLoader(train_sat_ds, num_workers=1)
val_sat_dl = DataLoader(val_sat_ds, num_workers=1)

/home/jupyter/chirps/positive/chirps5120.pt


In [27]:
train_sat_ds[0][0].shape

torch.Size([9, 1, 224, 224])

In [ ]:
#preprocess(train_ds[0][0][0, :, :, :])

In [ ]:
print('GPU disponible:' , torch.cuda.is_available())
if torch.cuda.device_count() >= 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")

In [28]:
class TrainModule():
  
  def __init__(self, model, opt, loss_func, save_path = ""):
    #self.params = params
    self.model = model
    self.opt = opt
    self.loss_func = loss_func

    self.fig = plt.figure(figsize=(15, 7))
    self.ax0 = self.fig.add_subplot(1, 2, 1) 
    self.ax1 = self.fig.add_subplot(1, 2, 2)
    
    self.save_path = save_path

    self.save_period = 1

    self.visualize_period = 1

  def get_data(self, train_ds, valid_ds, bs, n_cores):

    self.bs = bs

    self.train_dl =  DataLoader(train_ds,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=n_cores)
        
    self.val_dl =  DataLoader(valid_ds,
                          batch_size=bs * 2,
                          num_workers=n_cores)
    
  def set_optimizer(self, opt):
    self.opt = opt

  def set_loss_func(self, loss_func):
    self.loss_func = loss_func
  
  def loss_batch(self, model, loss_func, xb, yb, opt=None):
    y_hat = model(xb)
    loss = loss_func(y_hat, yb)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    #accu = self.accuracy(y_hat, yb)

    return loss.item(), len(xb) #, accu

  def accuracy(self, y_hat_b,yb):
    
    preds = torch.argmax(torch.softmax(y_hat_b.view(-1), dim = 1),dim=1)
    counts = (preds == yb)*1.0
    
    return torch.mean(counts)

  def bin_accuracy(self, y_hat_b,yb):
    
    count = (yb == (y_hat_b > 0.5))*1.0
    return torch.mean(count)

  def register(self, res_list):
    
    losses, nums = zip(*res_list)
    
    N = np.sum(nums)
    loss_mean = np.sum(np.multiply(losses, nums))/N
    loss_std = np.sqrt(np.sum(np.multiply((losses-loss_mean)**2, nums))/(N-1))
    
    return loss_mean, loss_std

  #def early_stoping(self, learning_data):

  def plot_curves(self, loss_data, metric_data, epoch):
      yt = loss_data['train_mean']
      yv = loss_data['val_mean']
      x = np.arange(0, len(yt), 1)
      self.ax0.cla()
      self.ax0.plot(x, yt, label='train loss')
      self.ax0.plot(x, yv, label='val loss')
      self.ax0.legend(loc="upper right")
      self.ax0.set_title("Loss at epoch: {}".format(epoch))

      ytm = metric_data['train_mean']
      yvm = metric_data['val_mean']
      xm = np.arange(0, len(yt), 1)
      self.ax1.cla()
      self.ax1.plot(xm, ytm, label='train accu')
      self.ax1.plot(xm, yvm, label='val accu')
      self.ax1.legend(loc="lower right")
      self.ax1.set_title("Accuracy at epoch: {}".format(epoch))

      display(self.fig)
      
      clear_output(wait = True)
    
  def load_learning_data(self, learning_data):

    self.learning_data = learning_data

  def save_checkpoint(self, model, optimizer, loss_data, metric_data,  epoch):

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'loss': loss,
            'loss_data': loss_data,
            'metric_data': metric_data
            }, self.save_path + "checkpoint_" + str(epoch))
    
  def load_model(self, model, path):

    model = torch.load(path)

    return model
    
  def save_best_model(self, model, optimizer, loss_data, metric_data, epoch):

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'loss': loss,
            'loss_data': loss_data,
            'metric_data': metric_data
            }, self.save_path + "best_model")
    
  def load_checkpoint(self, path):

      checkpoint = torch.load(path)
      self.model.load_state_dict(checkpoint['model_state_dict'])
      self.opt.load_state_dict(checkpoint['optimizer_state_dict'])
      epoch = checkpoint['epoch']
      loss_data = checkpoint['loss_data']
      metric_data = checkpoint['metric_data']

      self.loss_data = loss_data
      self.metric_data = metric_data

      return epoch, loss_data, metric_data


  def fit(self, 
        epochs,
        metric=None,
        only_print=True,
        print_leap = 1):
    
    print('GPU disponible:' , torch.cuda.is_available())
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using ", device)
    
    self.model = self.model.float()
    self.model.cuda()
    
    if torch.cuda.device_count() >= 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        self.model = nn.DataParallel(self.model)

    #
    

    print("Is model using cuda: ", next(self.model.parameters()).device)

    if metric is None:
        #metric = self.accuracy
        metric= self.bin_accuracy
        #metric = self.loss_func

    loss_data = pd.DataFrame(
        columns=['epoch', 'train_mean', 'train_std', 'val_mean', 'val_std'])
    
    metric_data = pd.DataFrame(
        columns=['epoch', 'train_mean', 'train_std', 'val_mean', 'val_std'])
    
    best_val_loss = np.inf
    print("Start")
    for epoch in tqdm(range(epochs)):

        # Entrenamiento -------------------------------------------------------
        train_res = []
        train_metric = []
        self.model.train()
        for xb, yb in tqdm(self.train_dl):
            #print("Aqui")
            # Actualizacion de parametros
            xb, yb = xb.cuda(), yb.cuda()
            #for i in range(len(xb)):
            #    xb[:,i, :, :, :] = preproces(xb[:, i, :, :, :])
            self.opt.zero_grad()
            self.loss_batch(self.model, self.loss_func, xb.float(), yb.float(), self.opt)

            # Evaluacion en entrenamiento
            with torch.no_grad():
              train_res.append(self.loss_batch(self.model, self.loss_func, xb.float(), yb.float()))
              train_metric.append(self.loss_batch(self.model, metric, xb.float(), yb.float()))

        # Validacion ----------------------------------------------------------
        # Para evaluar se puede utilizar un metrica de rendimiento
        self.model.eval()

        with torch.no_grad():
            val_res = [
                self.loss_batch(self.model, self.loss_func, xb.cuda().float(), yb.cuda().float()) for xb, yb in self.val_dl]
            val_metric = [
                self.loss_batch(self.model, metric, xb.cuda().float(), yb.cuda().float()) for xb, yb in self.val_dl]

        
        val_loss0, val_std0 = self.register(val_res)
        tra_loss0, train_std0 = self.register(train_res)
        
        
        #if epoch % print_leap == 0:
        #    print('Epoca:', epoch, '- val:', val_loss, '- train:', tra_loss)

        loss_data = loss_data.append(
            {
                'epoch': epoch,
                'train_mean': tra_loss0,
                'train_std': train_std0,
                'val_mean': val_loss0,
                'val_std': val_std0
            },
            ignore_index=True)
        
        val_loss1, val_std1 = self.register(val_metric)
        tra_loss1, train_std1 = self.register(train_metric)
        
        metric_data = metric_data.append(
            {
                'epoch': epoch,
                'train_mean': tra_loss1,
                'train_std': train_std1,
                'val_mean': val_loss1,
                'val_std': val_std1
            },
            ignore_index=True)
        

        if (epoch + 1) % self.save_period == 0:
            #print("saving checkpoint at epoch ", epoch)
            self.save_checkpoint(self.model, self.opt, loss_data, metric_data, epoch)

        if epoch > epochs/epochs and val_loss0 < best_val_loss:
            #print("saving best model at epoch ", epoch)
            self.save_best_model(self.model, self.opt, loss_data, metric_data, epoch)

        if epoch % self.visualize_period  == 0:
            self.plot_curves(loss_data, metric_data, epoch)

    if only_print:
        print('Proceso terminado')
        self.loss_data = loss_data
        self.metric_data = metric_data
    else:
        return loss_data, metric_data


In [29]:
cl_model = ExpandedCnnLSTM('resnet18', 1, 1, 512, 2, debug=False)

Using cache found in /home/jupyter/.cache/torch/hub/pytorch_vision_v0.6.0


In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
opt = optim.Adam(cl_model.parameters())

loss_func = torch.nn.BCELoss()

TM = TrainModule(cl_model, opt, loss_func)

TM.get_data(train_sat_ds, val_sat_ds, 128, 1)

TM.fit(40)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:186: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:574: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:1276.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


In [ ]:
TM.load_checkpoint("/content/best_model")

TM.loss_data